In [1]:
import numpy as np
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pysam
from liftover import get_lifter
from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans
import tabix
import myvariant

In [4]:
# Make dictionary of LFC (using counts per million normalisation etc) by uniq-ID

LFC_by_uniq={}
with open('/nfs/users/nfs_e/er10/SGE/data/Release_4/HK_new_method_LFC_R4.txt', 'r') as f:
    header=f.readline().strip().split('\t')
    uniq_col=header.index('uniq_key')
    D7_LFC_col=header.index('combined_LFC_d7')
    D11_LFC_col=header.index('combined_LFC_d11')
    D15_LFC_col=header.index('combined_LFC_d15')
    D21_LFC_col=header.index('combined_LFC_d21')
    lines=f.readlines()
    for line in lines:
        fields=line.strip().split('\t')
        uniq=fields[uniq_col].strip('""')
        D7_LFC = fields[D7_LFC_col].strip('""')
        D11_LFC = fields[D11_LFC_col].strip('""')
        D15_LFC = fields[D15_LFC_col].strip('""')
        D21_LFC = fields[D21_LFC_col].strip('""')
        LFC_by_uniq[uniq]=[D7_LFC, D11_LFC, D15_LFC, D21_LFC]
    

In [33]:
''' ML_category is as follows:
- clinical benign/likely benign variants and variant seen in GnomAD or UKBB = 0
- denovo, pathogenic/clinical pathogenic variants from ClinVar and DECIPHER + denovo variants from Kaplanis/Samocha et al (where not VUS in Decipher), and GEL = 1'''

data_X=[]
data_y=[]
with open('/nfs/users/nfs_e/er10/SGE/SGE_paper/Sup_Info//Supp_Table_5.txt', 'r')as f:
    header=f.readline().strip().split('\t')
    #print(header)
    uniq_col=header.index('SGE_oligo_name')
    MLcat_col=header.index('Category_for_RF')
    lines=f.readlines()
    for line in lines:
        ML_cat=fields[MLcat_col].strip()
        uniq=fields[uniq_col].strip('""')
        D7_LFC, D11_LFC, D15_LFC, D21_LFC = LFC_by_uniq[uniq]
        if ML_cat != 'NA':
            X=[uniq, ML_cat, D7_LFC, D11_LFC, D15_LFC, D21_LFC]
            data_X.append(X)
            data_y.append(int(ML_cat))

all_data_X=np.array(data_X)
all_data_y=np.array(data_y)

['Variant_category', 'D15LFC_GMM_C1z', 'D15LFC_GMM_C2z', 'D15LFC_GMM_cut_0.99', 'D15LFC_GMM_cut_0.9', 'D7_combined_LFC', 'D11_combined_LFC', 'D15_combined_LFC', 'D21_combined_LFC', 'DeltaDeltaG', 'protein_cons', 'recurrent_clinical_vars', 'ML_category', 'My_clusters_C1z_90', 'Time_cluster', 'C1z_90_gmm_p0', 'C1z_90_gmm_p1', 'SpliceAI_delta_score', 'Primary_consequence', 'revel', 'inverse_sift', 'numeric_polyphen', 'Cut_0.9', 'Cut_0.8', 'Cut_0.7', 'Cut_0.6', 'redundancy', 'counts_NF_U_F', 'Clinical_source', 'Clinical_ID', 'inheritance', 'pathogenicity', 'sex', 'pop_db', 'UKBB_male_female_count', 'UKKBB_AC', 'UKBB_AN', 'UKBB_AF', 'UKBB_AP', 'Gnomad_maf', 'Gnomad_AC', '""', 'X', 'uniq_key', 'Old_name', 'Seq', 'New_name', 'old_region', 'chrom', 'VCF_position', 'VCF_Ref', 'VCF_Alt', 'PAM_codon', 'species', 'assembly', 'gene_id', 'transcript_id', 'ref_chr', 'ref_strand', 'ref_start', 'ref_end', 'revc', 'ref_seq', 'pam_seq', 'vcf_alias', 'vcf_var_id', 'mut_position', 'ref', 'new', 'ref_aa', '

In [34]:
#Stratified sampling of test and train datasets to ensure that the representation of positive and negative variants is equal in each

# split into train/test sets with same class ratio
trainX, testX, trainy, testy = train_test_split(all_data_X, all_data_y, test_size=0.2, random_state=2, stratify=data_y)
# summarize
train_0, train_1 = len(trainy[trainy==0]), len(trainy[trainy==1])
test_0, test_1 = len(testy[testy==0]), len(testy[testy==1])
print('Training set: TN=%d, TP=%d, Testing set: TN=%d, TP=%d' % (train_0, train_1, test_0, test_1), '\n')

Training set: TN=432, TP=134, Testing set: TN=108, TP=34 



In [35]:
# Write the test and train datasets to file

with open('/nfs/users/nfs_e/er10/SGE/data/Release_4/MachineLearning/ML_input_test_train_data_D7D11D15.txt', 'w')as out:
    out_header='\t'.join(['test/train', 'uniq_ID', 'ML_cat', 'D7_LFC', 'D11_LFC', 'D15_LFC', 'D21_LFC'])+'\n'
    out.write(out_header)
    for item in trainX:
        uniq, ML_cat, D7_LFC, D11_LFC, D15_LFC, D21_LFC = item
        out_line='\t'.join(['train', uniq, ML_cat, D7_LFC, D11_LFC, D15_LFC, D21_LFC])+'\n'
        out.write(out_line)
    for item in testX:
        uniq, ML_cat, D7_LFC, D11_LFC, D15_LFC, D21_LFC = item
        out_line='\t'.join(['test', uniq, ML_cat, D7_LFC, D11_LFC, D15_LFC, D21_LFC])+'\n'
        out.write(out_line)
        

In [37]:
#Evaluate the different models

D15_LFCs=[]
uniq_IDs=[]
testy=[]
trainy=[]
model1_testX=[]
model1_trainX=[]
model2_testX=[]
model2_trainX=[]
model3_testX=[]
model3_trainX=[]
model4_testX=[]
model4_trainX=[]
model5_testX=[]
model5_trainX=[]
model6_testX=[]
model6_trainX=[]
model7_testX=[]
model7_trainX=[]
model8_testX=[]
model8_trainX=[]


with open('/nfs/users/nfs_e/er10/SGE/data/Release_4/MachineLearning/ML_input_test_train_data_D7D11D15.txt', 'r')as f:
    header=f.readline().strip().split('\t')
    print(header)
    ID_col=header.index('uniq_ID')
    test_train_col=header.index('test/train')
    ML_cat_col=header.index('ML_cat')
    D7_LFC_col=header.index('D7_LFC')
    D11_LFC_col=header.index('D11_LFC')
    D15_LFC_col=header.index('D15_LFC')
    D21_LFC_col=header.index('D21_LFC')
    lines=f.readlines()
    for line in lines:
        fields=line.strip().split('\t')
        uniq=fields[ID_col].strip()
        test_train = fields[test_train_col].strip()
        ML_cat=fields[ML_cat_col].strip()
        D7_LFC=float(fields[D7_LFC_col].strip())
        D11_LFC=float(fields[D11_LFC_col].strip())
        D15_LFC=float(fields[D15_LFC_col].strip())
        D21_LFC=float(fields[D21_LFC_col].strip())
        if test_train=='test':
            testy.append([int(ML_cat), uniq])
            model1_testX.append([D7_LFC,D11_LFC,D15_LFC])
            model2_testX.append([D7_LFC])
            model3_testX.append([D11_LFC])
            model4_testX.append([D15_LFC])
            model5_testX.append([D7_LFC,D11_LFC])
            model6_testX.append([D7_LFC, D15_LFC])
            model7_testX.append([D11_LFC, D15_LFC])
            model8_testX.append([D7_LFC, D11_LFC, D15_LFC, D21_LFC])
        elif test_train =='train':
            trainy.append(int(ML_cat))
            model1_trainX.append([D7_LFC,D11_LFC,D15_LFC])
            model2_trainX.append([D7_LFC])
            model3_trainX.append([D11_LFC])
            model4_trainX.append([D15_LFC])
            model5_trainX.append([D7_LFC,D11_LFC])
            model6_trainX.append([D7_LFC, D15_LFC])
            model7_trainX.append([D11_LFC, D15_LFC])
            model8_trainX.append([D7_LFC, D11_LFC, D15_LFC, D21_LFC])
            

['test/train', 'uniq_ID', 'ML_cat', 'D7_LFC', 'D11_LFC', 'D15_LFC', 'D21_LFC']


In [20]:
'''ML Models
1. D7_LFC,D11_LFC,D15_LFC
2. D7_LFC
3. D11_LFC
4. D15_LFC
5. D7_LFC,D11_LFC
6. D7_LFC, D15_LFC
7. D11_LFC, D15_LFC
8. D7_LFC, D11_LFC, D15_LFC, D21_LFC

Compare with using GMM posterior-probability cut_offs
'''

training_sets=[[model1_trainX, model1_testX], [model2_trainX, model2_testX], [model3_trainX, model3_testX], [model4_trainX, model4_testX], [model5_trainX, model5_testX], [model6_trainX, model6_testX], [model7_trainX, model7_testX], [model8_trainX, model8_testX]]

counter=1
for pair in training_sets:
    trainX, testX = pair
    TP, TN, FP, FN=(0,0,0,0)
    model = RandomForestClassifier()
    model = model.fit(trainX, trainy)
    print('model', counter)
    counter = counter+1

    x=0
    while x<len(testX):
        for row in testX:
            pred = model.predict([row])
            # GnomAD variants:
            if testy[x]==0:
                # This variant is a true negative, calculate the TN and FP rate using the model's prediction
                if pred[0]==0:
                    TN=TN+1
                elif pred[0]==1:
                    #print('false positive', row)
                    FP=FP+1

            # clinical variants
            elif testy[x]==1:
                if pred[0] == 1:
                    TP=TP+1
                elif pred[0] ==0:
                    #print('false negative', row)
                    FN=FN+1

            x=x+1

    '''Sensitivity = TP/TP+FN, PPV = TP/TP+FP, Specificity = TN/TN+FP, NPV = TN/(FN+TN)'''

    sens=TP/(TP+FN)
    spec=TN/(TN+FP)
    PPV=TP/(TP+FP)
    NPV=TN/(TN+FN)
    print('true positives', TP)
    print('true negatives', TN)
    print('false positives', FP)
    print('false negatives', FN)
    print('sensitivity', sens, 'specificity', spec, 'PPV', PPV, 'NPV', NPV, '\n')

model 1
true positives 33
true negatives 108
false positives 0
false negatives 1
sensitivity 0.9705882352941176 specificity 1.0 PPV 1.0 NPV 0.9908256880733946 



In [27]:
#Apply model to the rest of the data and predict classification for each variant:

model = RandomForestClassifier()
model = model.fit(model1_trainX, trainy)

with open('/nfs/users/nfs_e/er10/SGE/data/Release_4/MachineLearning/ML_output_R4_4_all_training_vars.txt', 'w') as out:
    out_header='\t'.join(['Uniq_ID', 'LFCD7', 'LFCD11', 'LFCD15','ML_category', 'prob_functional', 'prob_non_functional'])+'\n'
    out.write(out_header)
    for uniqID in LFC_by_uniq:
        LFCd7, LFCd11, LFCd15, LFCd21= LFC_by_uniq[uniqID]
        LFCd7=float(LFCd7)
        LFCd11=float(LFCd11)
        LFCd15=float(LFCd15)
        model_input = [[LFCd7, LFCd11, LFCd15]]
        yhat = model.predict(model_input)
        predicted_class = yhat[0]
        probs = model.predict_proba(model_input)
        for item in probs:
            prob_F = item[0]
            prob_NF = item[1]
        out_line = '\t'.join([uniqID, str(LFCd7), str(LFCd11), str(LFCd15), str(predicted_class), str(prob_F), str(prob_NF)])+'\n'
        out.write(out_line)
        